In [1]:
import torch.nn            as nn
import torch.optim         as optim
import numpy               as np
import matplotlib.pyplot   as plt
import torch.nn.functional as F
import networkx            as nx
import itertools
import torch

from libraries.graph      import POSCAR_graph_encoding, graph_POSCAR_encoding
from os                   import path, listdir
from torch.utils.data     import random_split
from torch_geometric.data import Data
from pymatgen.core        import Structure
from scipy.spatial        import Voronoi
from torch.nn             import Linear
from scipy.optimize       import minimize

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Graph embedding

In [2]:
temp_structure = Structure.from_file('POSCAR')

cell = np.array([
        [12.6, 0, 0],
        [0, 17.2, 0],
        [0, 0, 10.3]
    ])

n_atoms =  temp_structure.num_sites

In [3]:
distance_threshold = 10
y = 1
cell, composition, concentration, positions = MPL.information_from_VASPfile('.',
                                                                            'POSCAR')

nodes, edges, attributes = MPL.graph_POSCAR_encoding('0',
                                                    cell,
                                                    composition,
                                                    concentration,
                                                    positions,
                                                    distance_threshold=distance_threshold
                                                   )

#attributes = attributes * 100
temp = Data(x=nodes,
            edge_index=edges.t().contiguous(),
            edge_attr=attributes.flatten(),
            y=torch.tensor([y], dtype=torch.float)
           )
temp

Data(x=[71, 5], edge_index=[2, 4773], edge_attr=[4773], y=[1])

In [4]:
def update_n_connected_dict(n_connected_dict, idx_0, edge_indexes, edge_attributes):
    for i in np.arange(3+1)[::-1]:  # i = {3, 2, 1, 0}
        for idx_t in n_connected_dict[i]:
            if get_distance_attribute(idx_0, idx_t, edge_indexes, edge_attributes) is not None:
                # Remove from current list
                n_connected_dict[i].remove(idx_t)
                
                if i < 3:  # Else there is no list
                    # Append to next list
                    n_connected_dict[i+1].append(idx_t)
    return n_connected_dict


def get_n_connected(idx_0, cartesian_positions, edge_indexes, edge_attributes):
    n_connected = 0
    idx_connected = []
    for idx_t in list(cartesian_positions.keys()):
        if get_distance_attribute(idx_0, idx_t, edge_indexes, edge_attributes) is not None:
            n_connected += 1
            idx_connected.append(idx_t)
    
    return n_connected, idx_connected

def allocate_atom_n(d_01, x2, y2, d_0n, d_1n, d_2n):
    """Calculate the coordinates of atom 'n' based on geometric constraints.

    Args:
        d_01 (float): Distance between atoms '0' and '1'.
        x2   (float): x-coordinate of atom '2'.
        y2   (float): y-coordinate of atom '2'.
        d_0n (float): Distance between atoms '0' and 'n'.
        d_1n (float): Distance between atoms '1' and 'n'.
        d_2n (float): Distance between atoms '2' and 'n'.

    Returns:
        list: A list containing the x, y, and z coordinates of atom 'n'.
    """
    
    # Calculate x-coordinate of atom 'n'
    xn = (d_01**2 + d_0n**2 - d_1n**2) / (2 * d_01)
    
    # Calculate y-coordinate of atom 'n'
    yn = (d_1n**2 - d_2n**2 - d_01**2 + 2 * xn * d_01 + x2**2 - 2 * xn * x2 + y2**2) / (2 * y2)
    
    # Calculate z-coordinate of atom 'n'
    zn_square = d_0n**2 - xn**2 - yn**2
    if zn_square > -1e-4:  # Accounting for numerical errors
        zn = np.sqrt(zn_square)
        return [xn, yn, zn]
    return None


def get_distance_attribute(index0, index1, edge_indexes, edge_attributes):
    """Get the distance attribute between two nodes with given indices.

    Args:
        index0          (int):        Index of the first node.
        index1          (int):        Index of the second node.
        edge_indexes    (np.ndarray): Array containing indices of connected nodes for each edge.
        edge_attributes (np.ndarray): Array containing attributes corresponding to each edge.

    Returns:
        float: The distance attribute between the two nodes.
        False if index0, index1 not linked.
    """
    
    # Create a mask to find matching edges
    mask_direct  = (edge_indexes[0] == index0) & (edge_indexes[1] == index1)
    mask_reverse = (edge_indexes[0] == index1) & (edge_indexes[1] == index0)
    
    # Check if any edge satisfies the conditions
    matching_edge_indices = np.where(mask_direct | mask_reverse)[0]
    
    if len(matching_edge_indices) == 0:
        return None  # The pair is not linked
    
    # Get the distance attribute from the first matching edge
    distance_attribute = edge_attributes[matching_edge_indices[0]]
    
    return distance_attribute


def find_initial_basis(total_particles, edge_indexes, edge_attributes):
    for idx_0 in range(total_particles):
        for idx_1 in np.arange(idx_0+1, total_particles):
            for idx_2 in np.arange(idx_1+1, total_particles):
                condition_01 = (get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes) is not None)
                condition_12 = (get_distance_attribute(idx_1, idx_2, edge_indexes, edge_attributes) is not None)
                condition_02 = (get_distance_attribute(idx_0, idx_2, edge_indexes, edge_attributes) is not None)
                if condition_01 and condition_12 and condition_02:
                    return idx_0, idx_1, idx_2


def find_valid_reference(n_connected, idx_connected, edge_indexes, edge_attributes):
    for i in range(n_connected):
        for j in np.arange(i+1, n_connected):
            for k in np.arange(j+1, n_connected):
                idx_0 = idx_connected[i]
                idx_1 = idx_connected[j]
                idx_2 = idx_connected[k]

                x2, y2, _ = cartesian_positions[idx_2]
                
                # Get necessary distances
                d_01 = get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes)
                d_0n = get_distance_attribute(idx_0, idx,   edge_indexes, edge_attributes)
                d_1n = get_distance_attribute(idx_1, idx,   edge_indexes, edge_attributes)
                d_2n = get_distance_attribute(idx_2, idx,   edge_indexes, edge_attributes)
                
                temp_position = allocate_atom_n(d_01, x2, y2, d_0n, d_1n, d_2n)
                
                if temp_position is not None:
                    return temp_position

In [5]:
# Extract indexes and attributes from the graph
edge_indexes    = edges.T.detach().cpu().numpy().copy()
edge_attributes = attributes.detach().cpu().numpy().copy().flatten()

# Define the number of atoms in the graph
total_particles = len(nodes)

# Select three initial particles which are interconnected
idx_0, idx_1, idx_2 = find_initial_basis(total_particles, edge_indexes, edge_attributes)

# Get necessary distances
d_01 = get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes)
d_02 = get_distance_attribute(idx_0, idx_2, edge_indexes, edge_attributes)
d_12 = get_distance_attribute(idx_1, idx_2, edge_indexes, edge_attributes)

# Reference the first three atoms
x2 = (d_01**2 + d_02**2 - d_12**2) / (2 * d_01)
y2 = np.sqrt(d_02**2 - x2**2)

# Impose three particles at the beginning
cartesian_positions = {
    idx_0: [0,    0,  0],
    idx_1: [d_01, 0,  0],
    idx_2: [x2,   y2, 0]
}

all_idxs = np.delete(np.arange(total_particles),
                     [idx_0, idx_1, idx_2])

highest_n_explored = np.min(all_idxs)

# Initialized to 3 for three connections
n_connected_dict = {
    0: [],
    1: [],
    2: [],
    3: all_idxs.tolist()
}

In [6]:
idx_0, idx_1, idx_2

(0, 1, 2)

for idx_i in range(total_particles):
    for idx_j in np.arange(idx_i+1, total_particles):
        for idx_k in np.arange(idx_j+1, total_particles):
            cartesian_positions = {
                idx_0: [0,    0,  0],
                idx_1: [d_01, 0,  0],
                idx_2: [x2,   y2, 0]
            }
            
            all_idxs = np.delete(np.arange(total_particles),
                     [idx_0, idx_1, idx_2])
            
            for idx in all_idxs:
                a, _ = get_n_connected(idx, cartesian_positions, edge_indexes, edge_attributes)
                if a >= 3:
                    print(a)

In [7]:
while len(n_connected_dict[3]):  # Goes until all particles have been studied
    # Using a first-one-first-out approach
    idx = n_connected_dict[3][0]

    n_connected, idx_connected = get_n_connected(idx, cartesian_positions, edge_indexes, edge_attributes)
    print(idx, n_connected, idx_connected)

    # Remove idx from 3_connected_dict
    n_connected_dict[3].remove(idx)
    
    # Set idx in cartesian_positions or else add it to n_connected_dict
    if n_connected >= 3:
        # Extract the cartesian coordinates of idx
        temp_position = find_valid_reference(n_connected, idx_connected, edge_indexes, edge_attributes)

        # Check if there are enough particles able to locate idx (images make this step more difficult
        if temp_position is not None:
            # Generate temporal dictionary with the cartesian coordinates
            temp_dict = {
                idx: temp_position
            }

            # Update general dictionary with cartesian coordinates
            cartesian_positions.update(temp_dict)
    else:
        # n_connected_dict is updated adding idx where it belongs to
        n_connected_dict[n_connected].append(idx)

3 3 [0, 1, 2]
4 4 [0, 1, 2, 3]
5 5 [0, 1, 2, 3, 4]
6 6 [0, 1, 2, 3, 4, 5]
7 7 [0, 1, 2, 3, 4, 5, 6]
8 8 [0, 1, 2, 3, 4, 5, 6, 7]
9 9 [0, 1, 2, 3, 4, 5, 6, 7, 8]
10 10 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
11 11 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
12 11 [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11]
13 12 [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12]
14 13 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13]
15 14 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14]
16 15 [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
17 16 [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
18 17 [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
19 18 [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
20 19 [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
21 20 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
22 21 [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
23 22 [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,

In [11]:
for i in range(len(cartesian_positions)):
    print(cartesian_positions[i])

[0, 0, 0]
[8.606804, 0, 0]
[0.0786160728500923, 4.182790165840319, 0]
[8.604196512179348, 4.198015030069471, 0.10667069670511106]
[3.0706270511174445, 2.1512771163133357, 7.5107882118199525]
[5.510652766999476, 2.0914055244469574, 7.5347554881796315]
[2.277155746319634, 1.8222564325651684, 2.5758113491306602]
[6.394913163609171, 1.9367583024023343, 2.614619270558854]
[2.2388282478990673, 6.26174314051574, 2.384373636559819]
[6.412643018046687, 6.144563690018191, 2.864416523636099]
[2.2656660314161825, -2.0588526096422046, 2.4851097292391335]
[6.3917064563570305, -2.348619457303764, 2.2617092359866895]
[4.316644172419615, -0.06783588528337633, 2.537998778826176]
[4.229311146843505, -0.1934635452214272, 2.574498616568285]
[4.266878652411348, 4.127607797958813, 2.6774013241540033]
[4.294987751828163, 4.005133595698196, 2.8984611882402658]
[4.279157757877182, 8.348709233841841, 2.8172297502193615]
[4.319540937165159, 8.193503580825007, 3.211865890177834]
[2.170991988023945, 2.2133843179073

In [9]:
n_connected_dict

{0: [], 1: [], 2: [], 3: []}